# Amazon Bedrock AgentCore Gateway - セマンティック検索チュートリアル

### チュートリアルの詳細


| 情報                 | 詳細                                                                              |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルの種類   | 会話形式                                                                          |
| エージェントの種類     | 単一                                                                             |
| AgentCore サービス   | AgentCore Gateway、AgentCore Identity                                             |
| エージェントフレームワーク | Strands Agents                                                                |
| LLM モデル          | Anthropic Claude Sonnet 4                                                        |
| チュートリアルのコンポーネント | Strands Agent からの Lambda バックアップ AgentCore Gateway の作成と使用        |
| チュートリアルの分野   | 分野横断的                                                                        |
| 例の複雑さ           | 簡単                                                                              |
| 使用する SDK         | Amazon BedrockAgentCore Python SDK と boto3

### チュートリアルのアーキテクチャ
Amazon Bedrock AgentCore Gateway は、エージェントとそれらが対話するために必要なツールやリソースとの間の統一された接続性を提供します。Gateway はこの接続層で複数の役割を果たします：

1. **セキュリティガード**：Gateway は OAuth 認証を管理し、有効なユーザー/エージェントのみがツール/リソースにアクセスできるようにします。
2. **翻訳者**：Gateway は、Model Context Protocol (MCP) などの一般的なプロトコルを使用したエージェントのリクエストを API リクエストや Lambda 呼び出しに変換します。これにより、開発者はサーバーのホスティング、プロトコル統合の管理、バージョンサポート、バージョンパッチなどを行う必要がなくなります。
3. **コンポーザー**：Gateway は、複数の API、関数、ツールをエージェントが使用できる単一の MCP エンドポイントにシームレスに組み合わせることを可能にします。
4. **キーチェーン**：Gateway は、適切なツールで使用する適切な認証情報の挿入を処理し、エージェントが異なる認証情報セットを必要とするツールをシームレスに活用できるようにします。
5. **リサーチャー**：Gateway は、エージェントが全てのツールを検索して、特定のコンテキストや質問に最適なツールのみを見つけることを可能にします。これにより、エージェントは少数ではなく何千ものツールを利用できるようになります。また、エージェントの LLM プロンプトに提供する必要があるツールのセットを最小限に抑え、レイテンシーとコストを削減します。
6. **インフラストラクチャマネージャー**：Gateway は完全にサーバーレスであり、組み込みの可観測性と監査機能を備えているため、開発者はエージェントとツールを統合するための追加のインフラストラクチャを管理する必要がありません。

![仕組み](images/gw-arch-overview.png)

### チュートリアルの主な機能

* AWS Lambda バックエンドのターゲットを持つ Amazon Bedrock AgentCore Gateway の作成
* AgentCore Gateway のセマンティック検索の使用
* Strands Agents を使用して AgentCore Gateway 検索がレイテンシーをどのように改善するかを示す

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents

# AgentCore Gateway は、多数のツールを持つ MCP サーバーの課題解決に役立ちます
一般的な企業環境では、エージェント開発者は数百、あるいは数千もの MCP ツールを持つ MCP サーバーに遭遇します。このような大量のツールは、ツール選択の精度低下、コスト増加、過剰なツールメタデータによるトークン使用量の増加に起因する高いレイテンシーなど、AI エージェントにとって課題をもたらします。
これは、エージェントをサードパーティのサービス（Zendesk、Salesforce、Slack、JIRA など）や既存の企業 REST サービスに接続する際に発生する可能性があります。
AgentCore Gateway はツール間の組み込みセマンティック検索を提供し、エージェントに必要なツールを与えながら、エージェントのレイテンシー、コスト、精度を向上させます。
ユースケース、LLM モデル、エージェントフレームワークによっては、典型的な MCP サーバーから数百のツールをすべて提供するのではなく、関連するツールにエージェントを集中させることで、最大 3 倍のレイテンシー改善が見られる場合があります。

![どのように機能するか](images/gateway_tool_search.png)

## このノートブックで学ぶこと
このノートブックでは、AgentCore Gateway 検索のチュートリアルを提供します。このステップバイステップのチュートリアルを終えると、以下のことが理解できるようになります：

- 関連ツールを素早く見つけるための AgentCore Gateway の組み込み検索ツールの使用方法
- ツール検索結果を Strands Agents に統合してレイテンシーを改善しコストを削減する方法

## ノートブック構造の概要
このノートブックは以下のセクションで構成されています：

1. AgentCore Gateway 検索の基本原理の理解
2. ノートブック環境の準備
3. 数百のツールを持つ Gateway のセットアップ
4. Gateway からのツール検索
5. 多数のツールを持つ MCP サーバーでの Strands Agents の使用
6. ツール検索結果を Strands Agent に追加する
7. ツール検索を使用して 3 倍のレイテンシー改善を示す

# AgentCore Gateway 検索の基本を理解する

AgentCore Gateway を作成する際、検索を有効にするオプションがあります。
検索が有効化された Gateway では、次の 3 つのことが起こります：

1. **ベクトルストアが作成される**。Gateway サービスは、新しい Gateway 用にサーバーレスで完全に管理されたベクトルストアを自動的に作成します。これにより、Gateway ツール全体にわたる完全な意味検索が可能になります。
3. **ベクトルストアにデータが格納される**。Gateway に Gateway Target を追加すると、サービスは自動的にバックグラウンドで埋め込みを使用して、新しい Target からのツールに基づいてベクトルストアにデータを格納します。ツールのメタデータは、ツールの JSON 定義または REST サービスターゲットの OpenAPI スキーマ仕様から取得されます。
2. **検索ツール（MCP ベース）が提供される**。ユーザー定義のすべてのツール（AWS Lambda ターゲットまたは REST サービス）に加えて、Gateway は意味検索を提供する追加の MCP ツールを 1 つ取得します。これは `x-amz-bedrock-agentcore-search` という名前です。このプレフィックスにより、ユーザー定義のツールとの名前の衝突がなくなります。将来的には、同様のツールをさらに追加する可能性もあります。検索ツールには `query` という単一の引数があります。検索ツールが呼び出されると、Gateway サービスはそのクエリを使用して意味検索を実行し、利用可能なツールのメタデータ（名前、説明、入力および出力スキーマ）と照合して、関連性の高い順に最も関連性の高いツールを返します。

# ノートブック環境の準備

In [ ]:
#!uv add -r requirements.txt

import os
os.environ['AWS_PROFILE'] = 'cline2'

必要な Python ライブラリをすべてインポートし、環境変数を読み込む

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

ロガーを設定する

In [ ]:
# ルートストランドロガーを設定する
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# ログを確認するためのハンドラーを追加する
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

boto3 バージョンを確認する

In [ ]:
boto3.__version__

AgentCore コントロールプレーン API 用の boto3 クライアントを取得します。

In [ ]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

# 数百のツールを持つゲートウェイのセットアップ

AgentCore Gateway は、既存の API を厳選されたセットとして MCP ツールとしてエージェントに公開するための安全でスケーラブルな方法を提供します。本番環境では、Gateway リソースは CloudFormation、CDK、または Terraform などのツールを使用したインフラストラクチャ・アズ・コードで作成されるでしょう。このチュートリアルでは、リソースと API をより効果的に理解できるように、boto3 コントロールプレーン API を直接使用します。これにより、独自のゲートウェイの構築と使用、そしてより強力で安全なエージェントの作成をより簡単に始めることができます。

高レベルでは、Gateway のセットアップ手順は次のとおりです：

1. インバウンド（エージェントが Gateway を呼び出す）とアウトバウンド（Gateway がツールを呼び出す）のセキュリティに使用するアイデンティティプロバイダーと認証情報プロバイダーを定義します。
2. `create_gateway` を使用して Gateway を作成します。
3. `create_gateway_target` を使用して Gateway ターゲットを追加し、AWS Lambda または既存の RESTful サービスで実装される MCP ツールを公開します。

このチュートリアルでは、アイデンティティプロバイダー（IdP）として Amazon Cognito を、ターゲットとして AWS Lambda 関数を、そしてアウトバウンド認証に AWS IAM を使用します。このチュートリアルで示される概念は、他の IdP や他のターゲットタイプを使用する場合にも同様に適用されます。

### Amazon Cognito リソースの作成

このチュートリアルでは、以下のリソースをすでに作成し、対応する環境変数を設定していることを前提としています：

- AWS Lambda 実行用の IAM ロール (`gateway_lambda_iam_role`)
- シンプルな数学ツール用の AWS Lambda 関数 (`calc_lambda_arn`)
- レストラン予約ツール用の AWS Lambda 関数 (`restaurant_lambda_arn`)
- Amazon Cognito ユーザープール（クライアント ID (`cognito_client_id`) とディスカバリー URL (`cognito_discovery_url`) を提供）

レストラン API の JSON ツールメタデータを見てみましょう。既存の REST サービスと統合する場合、API 仕様は代わりに OpenAPI Schema を使用して提供されることに注意してください。

In [ ]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

以下は簡単な計算機 API です。

In [ ]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

以下は電卓ツール用の AWS Lambda 関数の実装です。

In [ ]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

In [ ]:
#### AWS Lambda 関数のサンプルを作成し、MCP ツールに変換する
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            calc_lambda_resp["lambda_function_arn"],
        )

In [ ]:
calc_lambda_resp["lambda_function_arn"]

In [ ]:
#### AWS Lambda 関数のサンプルを作成し、それを MCP ツールに変換する
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

In [ ]:
restaurant_lambda_resp["lambda_function_arn"]

In [ ]:
cognito_response = utils.setup_cognito_user_pool()

In [ ]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

In [ ]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

#### コントロールプレーン API を使用するためのヘルパー関数をいくつか作成しましょう

In [ ]:
def read_apispec(json_file_path):
    try:
        # JSON ファイルを読み込み、内容を文字列として返す
        with open(json_file_path, "r") as file:
            # JSON を Python オブジェクトに解析する
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

#### Gateway ヘルパー関数の作成
ここでは、名前と説明を指定して AgentCore Gateway を作成するためのヘルパー関数を紹介します。この関数は IdP として Amazon Cognito を使用し、許可されたクライアント ID とディスカバリー URL を環境変数から取得します（これらは既に定義されています）。また、作成される Gateway ではデフォルトでセマンティック検索を有効にし、事前定義された IAM ロールを使用します。

In [ ]:
def create_gateway(gateway_name, gateway_desc):
    # Gateway へのインバウンド OAuth に Cognito を使用する
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # ツールのセマンティック検索を有効にする
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # ゲートウェイを作成する
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )

    return response["gatewayId"]

#### ゲートウェイターゲットヘルパー関数の作成
この関数は、既存のゲートウェイ上に新しい AWS Lambda ターゲットを作成します。
ゲートウェイ ID、新しいターゲットの名前と説明、既存の AWS Lambda 関数の ARN、そしてゲートウェイから公開したいツールのインターフェースを記述する JSON スキーマを提供するだけです。

In [ ]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Lambda ターゲットをゲートウェイに追加する
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # IAM を認証情報プロバイダーとして使用する
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

### 最初の AgentCore Gateway の作成
最初の Gateway をセットアップする前に、Gateway がどのように MCP ツールを使用するための受信リクエストと、Gateway からツールやリソースへの送信アクセスの両方に対してセキュリティを提供するかを簡単に見てみましょう。

![どのように機能するか](images/gateway_secure_access.png)

では、このチュートリアル用の gateway を作成しましょう。名前と説明を提供します。

In [ ]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

### AgentCore Gateway ターゲットの追加

このチュートリアルでは、すでに 2 つの Lambda 関数をインストールしていることを前提としています。1 つは単純な数学計算を行うもの、もう 1 つはレストラン予約の作成をシミュレートするものです。これらの関数それぞれに Gateway ターゲットを追加します。

これらのターゲットを追加した後、さらに追加のターゲットを加えて、より多くの MCP ツール数を実現し、AgentCore Gateway 検索の能力を実証するのに役立てます。

ゲートウェイが作成されたので、Lambda 関数を介してレストランの予約を行うためのターゲットを追加しましょう。

In [ ]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

ここでは 2 つ目のターゲットを追加します。今回は 4 つの基本的なツール（加算、減算、乗算、除算）を実装する Lambda と、投資管理用に生成された 75 個のツール定義（取引、信用調査、定量分析、ポートフォリオ管理）のセットを使用します。投資管理ツールの定義は実際には Lambda 関数内に実装されていません。これらは大量のツールを示すためだけに追加しています。

In [ ]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

ゲートウェイ検索の力を示すために、Calculator ターゲットをさらにいくつか追加して、300 個以上の MCP ツールが公開されるようにします。

In [ ]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [ ]:
add_more_tools(gatewayId=gatewayId)

In [ ]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

# Gateway からツールを検索する

### MCP リスト ツールに慣れる前に検索を行う

特定の Gateway ID に対する MCP エンドポイント URL を取得するため、また Gateway を安全に使用するための JWT OAuth アクセストークンを取得するためのユーティリティ関数をいくつか定義しましょう。

In [ ]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

今、私たちの Gateway が作成され、ターゲットを持っているので、その Gateway への MCP URL を取得しましょう。Gateway ID に基づいて Gateway コントロールプレーンからエンドポイント URL を取得することができます。

#### Gateway に対する MCP Inspector の使用

MCP サーバーのエンドポイント URL と JWT ベアラートークンを取得したので、MCP Inspector ツールを使用して MCP サーバーを探索したいかもしれません。MCP Inspector は、任意の MCP サーバーに接続でき、提供されているツールを一覧表示し、さらに使いやすいツール呼び出し体験を提供するオープンソースツールです。

ターミナルウィンドウから、単に `npx @modelcontextprotocol/inspector` と入力して MCP Inspector を起動します。次に、Gateway エンドポイント URL と JWT トークンを貼り付けて接続します。接続したら、「List Tools」と「Invoke Tool」を試してみてください。

以下はサンプルのスクリーンショットです。

![MCP Inspector](images/mcp_inspector.png)

In [ ]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

MCP サーバーセキュリティは OAuth に基づいています。私たちの Gateway と対話するためには、IdP から JWT OAuth アクセストークンを取得する必要があります。

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

In [ ]:
# !npx @modelcontextprotocol/inspector

#### MCP ツールを呼び出したり一覧表示したりするための jsonrpc を使用するヘルパー関数の作成
`invoke_gateway_tool` という名前のヘルパー関数を定義しましょう。これは jsonrpc を使用して、MCP サーバーによって公開されている任意の MCP ツールを呼び出すためのものです。もちろん、あなたの Gateway も含まれます。エンドポイント URL と JWT トークンがあれば、このユーティリティを使用して、Gateway に Gateway ターゲットを追加したときに AgentCore Gateway が利用可能にした任意の MCP ツールを呼び出すことができます。

In [ ]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):

    # print(f"ツール {tool_params['name']} を呼び出しています")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

こちらは Gateway から利用可能な MCP ツールをリストアップするための MCP の `tools/list` メソッドを使用するためのもう一つのユーティリティ関数です。Gateway ID と JWT トークンを与えると、その Gateway から完全なツールセットを取得し、エージェントですぐに使える形式でリストを返します。返されるリストには、あなたのエージェントに渡すのに適した Strands Agents MCPAgentTool オブジェクトが含まれています。

`tools/list` 呼び出しはページ分割されているため、この関数はループして一度に 1 ページのツールを取得し、`nextCursor` フィールドが設定されなくなるまで続ける必要があります。このユーティリティ関数は HTTPS と jsonrpc プロトコルを使用してエンドポイントを直接呼び出します。これは Strands Agents が提供する `MCPClient` クラスと比較して、ツールをリストアップするためのより低レベルな方法です。この体験については後ほど見ていきます。

In [ ]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

この補助関数を使用して結果を確認しましょう。

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")

#### Strands Agents の list_tools_sync() をページネーションで使用する
Python ベースの MCP クライアントを書いたことがあれば、クライアントが関連付けられている MCP サーバーから利用可能なツールのセットを返す `list_tools_sync()` メソッドに馴染みがあるでしょう。しかし、MCP の list tools もページネーションされていることをご存知でしたか？ デフォルトでは、最初の小さなサブセットのツールのみが返されます。シンプルな MCP サーバーでは気づかなかったかもしれませんが、多くの実際の MCP サーバーでは、ページが残っていなくなるまで、一度に 1 ページずつツールを取得するループが必要です。以下のユーティリティ `get_all_mcp_tools_from_mcp_client` はまさにそれを行います。これは指定された Strands Agent MCP クライアントからツールの完全なリストを返します。

In [ ]:
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Gateway を使って Python クライアントが見つけるツールの数を確認してみましょう。
まず、エンドポイント URL と JWT ベアラートークンに基づいて MCPClient オブジェクトを作成します。次に、
MCP サーバーから返される多くのページにわたるツールの完全なセットを取得します。
先ほど追加したターゲットを考慮すると、300 以上のツールが返されるはずです。

In [ ]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

私たちは、Gateway を MCP サーバーとして使用して、Gateway からツールの完全なセットを取得する 3 つの異なる方法を見てきました：

1. jsonrpc を直接使用する
2. Strands Agent MCPClient の `list_tools_sync()` メソッドを使用する
3. MCP Inspector ツールを使用する（これは裏側で jsonrpc を使用しています）

エージェントを構築する一般的な開発者の場合、オプション 2 を使用することになります。

### 組み込みのゲートウェイセマンティック検索ツールの使用
ここで、MCP ツールリストに追加される追加の MCP ツールとして提供されている組み込みの検索ツールを使用して、ゲートウェイでの最初のセマンティック検索を試してみましょう。

まず、MCP を使用して検索ツールを実行するためのシンプルなユーティリティ関数を定義しましょう。
ツールの一覧表示と同様に、ゲートウェイエンドポイントと JWT トークンが必要です。それ以外には、
検索クエリを渡すだけです。Gateway 検索ツールが残りの処理を行い、
自動的に管理しているサーバーレスベクトルストアに対してそのクエリを照合します。

In [ ]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [ ]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

検索がほとんどの場合 1 秒以内に返ってくる速さに注目してください。結果はクエリとツールのメタデータのマッチングに基づく検索関連性の降順で返されます。最も関連性の高いツールがリストの最初に表示されます。検索の初期実装では最大 10 件の結果が返されます。これらのツールをすべてエージェントで使用することも、最も関連性の高い一部のマッチだけを選ぶこともできます。

# MCP サーバーに多数のツールがある場合の Strands Agents の使用

まず、Strands Agent で使用するモデルを選択します。
このノートブックでは、Amazon Bedrock モデルを使用していますが、Strands と AgentCore
はどの LLM でも動作することができます。

In [ ]:
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### シンプルな Strands Agent を使用して AgentCore Gateway のエージェントツールを活用する
ここでは、Strands Agent を使用して AgentCore Gateway が提供する MCP サーバーを活用する方法がいかに簡単かを示します。この
シンプルな例では、エージェントにいくつかの数字を足すように依頼します。

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")

Strands Agents フレームワークでは、エージェントのイベントループをバイパスして、MCP ツールを直接呼び出すこともできます。
Gateway ツールはネイティブの MCP ツールとして公開されているため、Gateway ツールに対しても同様に行うことができます。ここでは
`agent.tool.<tool_name>(args)` 構文を使用して Gateway MCP ツールを呼び出します：

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

In [ ]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [ ]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))
    return search_results

In [ ]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

In [ ]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

# Strands Agent へのツール検索結果の追加

ここでは、検索から返されたツールを Strands Agent に追加する方法を見ていきましょう。コーディングを簡単にするために、ツール検索結果を Strands MCPAgentTool オブジェクトにマッピングするユーティリティ関数を提供しましょう。単に検索結果を渡し、それらの結果のうち何個をエージェントに渡したいかを指定するだけです。

In [ ]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")

レイテンシの改善に注目してください。Gateway 検索からのツールのサブセットを使用したこの例は、何百ものツールに依存する場合、エージェント呼び出しよりも大幅に高速です。

# ツール検索を使用して 3 倍のレイテンシ改善を示す

# 日本語訳:

Strands エージェントから Gateway MCP ツールを使用する方法と、ツールを検索してエージェントに追加する方法がわかったところで、検索の力を示しましょう。実現できる大幅なレイテンシー削減と入力トークン使用量の削減について説明します。

レイテンシーとトークン削減を実証するために、2 つのアプローチを並べて比較します：

1. **検索なし**。MCP サーバーが公開する MCP ツールの完全なセット（私たちの場合は 300 以上）をエージェントに追加し、エージェントにツールの選択と呼び出しを任せます。
2. **検索を使用**。2 つ目のアプローチでは、対象となるトピックに基づいて検索を行い、最も関連性の高いツールのみをエージェントに送ります。この点を証明するために、数学（数字の足し算）と食べ物（レストラン予約）という 2 つの異なるトピックを使用し、それぞれ異なるツールセットを必要とします。

レイテンシー分布を正規化し、意味のある比較を得るために、各アプローチで複数の反復を実行します。また、効果を過大評価しないために、検索アプローチを行う際には、エージェント呼び出しのレイテンシーだけでなく、ツール検索を実行するレイテンシーも含めています。各反復で、エージェントに 2 つのタスクを与えます：

1. 数学タスク - 2 つの数字を足す
2. 食べ物タスク - レストランの予約をする

以下の結果は、3 倍のレイテンシー削減と、さらに大きな入力トークン使用量の削減という利点を示しています。トークン使用量の節約はコスト削減につながりますが、入力トークンの相対的に低いコストのため、それほど大きな影響がない場合があることに注意してください（多くのモデルプロバイダーでは、入力トークンははるかに安価です）。それでも、大規模なエージェントデプロイメントでは、入力トークン使用コストも積み重なるため、動的検索はエージェントの実行時コストの削減にも役立ちます。

#### MCP ツールの全セットを使用するエージェントの遅延とトークン使用量を測定する

In [ ]:
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

In [ ]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")

#### エージェントの Gateway Search を使用したレイテンシーとトークン使用量の測定
ここでは動的なアプローチを使用し、検索を呼び出して関連ツールを見つけ、その後、それらの関連ツールのみを使用してエージェントを呼び出します。各会話ターンでエージェントをリセットしているため、前のターンの会話履歴からメッセージリストも初期化していることに注意してください。

In [ ]:
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

#### 結果を比較し、検索の利点を強調する

In [ ]:
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")

# 結論
このチュートリアルでは、Amazon Bedrock AgentCore Gateway とその組み込みの完全マネージド型セマンティック検索機能について学びました。以下の内容を見てきました：

- セマンティック検索を有効にしたゲートウェイの作成方法
- 単一のエンドポイントから 300 以上の MCP ツールを提供するための複数のゲートウェイターゲットの追加方法
- 3 つの異なるアプローチを使用してゲートウェイ上のツールを一覧表示する方法
- 関連ツールを見つけるための組み込みセマンティック検索ツールの使用方法
- 検索と Strands Agent の統合方法
- 何百ものツールを使用するサーバーを使用するエージェントと、セマンティック検索を使用して特定のトピックにツールを絞り込むエージェントのパフォーマンスを比較する方法

AgentCore Gateway の検索は、より高度なユースケースにも役立ちます。検索をコントロールプレーン API だけでなくネイティブな MCP ツールとして提供することで、エージェントに新しい MCP サーバーを発見し、実行時に新しい機能を見つける自律性を与えることができ、より困難な問題を解決するためのブレークスルーにつながることが想像できます。さらに、検索は MCP レジストリの重要な基盤であり、エージェント開発者が新しいエージェントを設計・構築する際のサポートとなります。

# リソースのクリーンアップ

AgentCore Gateway リソースをクリーンアップするためのヘルパー関数をいくつか定義しましょう。

In [ ]:
def delete_gatewaytarget(gateway_id):
    response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

    print(f"Found {len(response['items'])} targets for the gateway")

    for target in response["items"]:
        print(
            f"Deleting target with Name: {target['name']} and Id: {target['targetId']}"
        )

        response = agentcore_client.delete_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target["targetId"]
        )
        time.sleep(20)


def delete_gateway(gateway_id):
    response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

### ゲートウェイターゲットの削除

In [ ]:
delete_gatewaytarget(gateway_id=gatewayId)

### ゲートウェイ自体の削除

In [ ]:
delete_gateway(gateway_id=gatewayId)

In [ ]:
lambda_arns = [
    calc_lambda_resp["lambda_function_arn"],
    restaurant_lambda_resp["lambda_function_arn"],
]

for arn in lambda_arns:
    if utils.delete_gateway_lambda(arn):
        print(f"Deleted Lambda: {arn}")
    else:
        print(f"Lambda {arn} not found or deletion failed")

In [ ]:
# Gateway role cleanup

# ゲートウェイ ロールのクリーンアップ
if utils.delete_gateway_iam_role():
    print("Gateway IAM role deleted")
else:
    print("Gateway IAM role not found or deletion failed")

# Cognito クリーンアップ
if utils.delete_cognito_user_pool():
    print("Cognito pool deleted")
else:
    print("✗ Failed to delete Cognito pool")